In [1]:
from ast import arg
import sys, os
sys.path.append(os.getcwd())
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from utils.dist_utils import init_distributed_mode

from model import ACTIONCHORE
from trainer.trainer import Trainer
from data.data_paths import DataPaths
from data.train_data import BehaveDataset


from config.config_loader import load_configs


configs = load_configs('chore-release')
# assert args.exp_name==configs.exp_name
# add command line configs
configs.device = 'cuda'
configs.world_size = 4
configs.dist_url = 'env://'
configs.rank = 0

def index(feat, uv):
    '''
    :param feat: [B, C, H, W] image features
    :param uv: [B, 2, N] uv coordinates in the image plane, range [-1, 1]
    :return: [B, C, N] image features at the uv coordinates
    '''
    uv = uv.transpose(1, 2)  # [B, N, 2]
    uv = uv.unsqueeze(2)  # [B, N, 1, 2]
    samples = torch.nn.functional.grid_sample(feat, uv, align_corners=True)  # [B, C, N, 1] or [B, C, N, 5]

    return samples[:, :, :, 0]  # [B, C, N]

configs loaded from config/chore-release.json


In [2]:
def test(args):
    world_size = torch.cuda.device_count()
    init_distributed_mode(args)
    rank = args.rank
    device = torch.device(args.device)
    model = ACTIONCHORE(args, rank=rank)

    model.check_action_chore()

#     batch_size = 1
#     # test images 
#     # net_img_size = 512, 512
#     # batch_size = 15
#     test_img = torch.randn((batch_size, 5, 512, 512))  # [B, C, H, W] 
#     im_feat_list, tmpx, normx = model.filter(test_img)
    
#     # fake points
#     points = torch.randn((batch_size, 100, 3)) # [B, N, 3]
#     xyz = points.permute(0, 2, 1) # [B, 3, N]
#     xy = xyz[:, :2, :]              # [B, 2, N]
#     rela_z = (points[:, :, 2:3] - 2.2).transpose(1, 2)  # relative depth to a fixed smpl center
#     z_feat = torch.cat([points[:, :, 0:2].transpose(1, 2), rela_z], 1)
#     tmpx_local_feature = index(tmpx, xy)


#     for im_feat in im_feat_list:
#             point_local_feat_list = [index(im_feat, xy), z_feat]
#             point_local_feat_list.append(tmpx_local_feature)

#             # I need to add action feature here

#             point_local_feat = torch.cat(point_local_feat_list, 1)
#             # preds = decode(point_local_feat)

#             # # out of image plane is always set to a maximum
#             # df = preds[0]  # the first is always df prediction
#             # df_trans = df.transpose(1, 2)  # (B, 2, N) -> (B, N, 2)
#             # df_trans[~in_img] = self.OUT_DIST
#             # df = df_trans.transpose(1, 2)

#     return (point_local_feat_list, point_local_feat)

test(configs)